<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Download-the-data" data-toc-modified-id="Download-the-data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Download the data</a></span></li><li><span><a href="#Some-preprocession-magic" data-toc-modified-id="Some-preprocession-magic-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Some preprocession magic</a></span></li><li><span><a href="#Apply-some-fucking-model" data-toc-modified-id="Apply-some-fucking-model-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Apply some fucking model</a></span></li></ul></div>

## Download the data

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [3]:
tx_data = pd.read_csv('tax-suggestions-ru.csv')
print(tx_data.shape)
tx_data.head()

(16695, 3)


,MARKETPLACE,SEARCHTERMS,TYPE
0,NaN,Серена Уильямс,TAXONOMY
1,NaN,Расселл Уэстбрук,TAXONOMY
2,NaN,Леброн Джеймс,TAXONOMY
3,NaN,Мишель Ви,TAXONOMY
4,NaN,Яннис Адетокумбо,TAXONOMY


In [4]:
for i in range(tx_data.shape[1]):
    print(tx_data.iloc[:, i].value_counts(), '\n')

Series([], Name: MARKETPLACE, dtype: int64) 

Твердый грунт Спортивный стиль Шлепанцы    2
Мальчики Спортивный стиль Шлепанцы         2
Девочки Кожа Шлепанцы                      2
Мужчины Nike Lunarlon Шлепанцы             2
На стадионе Крикет Шлепанцы                2
                                          ..
Баскетбол                                  1
Школьники Чирлидинг Платья                 1
Сиэтл Сихокс Форма                         1
По дороге Скейтбординг Мячи                1
Dri-FIT Перчатки и варежки                 1
Name: SEARCHTERMS, Length: 15058, dtype: int64 

TAXONOMY    16695
Name: TYPE, dtype: int64 



In [5]:
# other cols are pretty useless
data = tx_data.iloc[:, 1]

## Some preprocession magic

In [17]:
data_list = data.values.tolist()
data_list = [el for el in data_list]
data_list

['Серена Уильямс',
 'Расселл Уэстбрук',
 'Леброн Джеймс',
 'Мишель Ви',
 'Яннис Адетокумбо',
 'Кевин Дюрант',
 'Наоми Осака',
 'Лике Мартенс',
 'Тайгер Вудс',
 'Килиан Мбаппе',
 'Джейсон Дэй',
 'Пол Джордж',
 'Кайри Ирвинг',
 'Рори Макилрой',
 'Криштиану Роналду',
 'Найджа Хьюстон',
 'Рафаэль Надаль',
 'Холодная погода',
 'Теплая погода',
 'Ам. футбол (беки и ресиверы)',
 'Nike Sportswear',
 'ACG',
 'Jordan',
 'Nike в твоем стиле',
 'Nike',
 'NikeLab',
 'Чили',
 'Южная Корея',
 'Великобритания',
 'Турция',
 'Словакия',
 'Норвегия',
 'Португалия',
 'Страны',
 'Испания',
 'Греция',
 'Нидерланды',
 'Франция',
 'США',
 'Австралия',
 'Хорватия',
 'Словения',
 'Канада',
 'Кения',
 'Польша',
 'Саудовская Аравия',
 'ЮАР',
 'Китай',
 'Нигерия',
 'Англия',
 'Россия',
 'Бразилия',
 'Тканый материал',
 'Сетка',
 'Плотное прилегание',
 'Свободный крой',
 'Прилегающий',
 'Стандарт',
 'Девочки',
 'Мужчины',
 'Мальчики',
 'Женщины',
 'Air Max 1',
 'Jordan 5',
 'Huarache',
 'Nike SuperRep',
 'Air Max 9

In [13]:
'''
Насколько я понимаю, в этих данных главная особенность в том что сначала (вид товара на русском), потом (модель на англ)
'''

def string_in_charset(s: str, chrset: set):
    # may be optimized duno
    return set(s).issubset(chrset)

def latin_nonlatin_split(s: str):
    # tokenize
    tokens = s.split(' ')
    # define two sets of symbols for two parts of 'sentence'
    latin_plus_nums = set([chr(ind) for ind in range(65, 91)] 
                    + [chr(ind) for ind in range(97, 122)] 
                    + [chr(ind) for ind in range(48, 58)])
            
    non_latin_chars = set([chr(ind) for ind in range(ord('а'), ord('я')+1)] 
                    + [chr(ind) for ind in range(ord('А'), ord('Я')+1)])
    # store split data as two lists, every element structured like (original_index, value)
    latin = [(ind, tokens[ind]) 
             for ind in range(len(tokens)) 
             if string_in_charset(tokens[ind], latin_plus_nums)
            ]
    
    non_latin = [(ind, tokens[ind]) 
             for ind in range(len(tokens)) 
             if string_in_charset(tokens[ind], non_latin_chars)
                ]
    
    return latin, non_latin

In [16]:
latin_nonlatin_split('Верх Internationalist')

([(1, 'Internationalist')], [(0, 'Верх')])

In [21]:
# a list of tuples (latin, non-latin)
data_proc = [latin_nonlatin_split(rec) for rec in data_list]
data_proc[:3]

[([], [(0, 'Серена'), (1, 'Уильямс')]),
 ([], [(0, 'Расселл'), (1, 'Уэстбрук')]),
 ([], [(0, 'Леброн'), (1, 'Джеймс')])]

## Apply some fucking model
We don't really need to do anything to english words

In [22]:
#pip install pymorphy2
#pip install -U pymorphy2-dicts-ru

In [25]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer(lang='ru')

In [29]:
result_non_latin = []
for record in data_proc:
    new_record = [(elem[0],
                    morph.parse(elem[1])[0].normal_form)
                    for elem in record[1]
                 ]
    result_non_latin.append(new_record)
    
print(result_non_latin)
    
    
    #for ind in range(len(record[1])):
    #    word2proc = record[1][ind]
    #    print(word2proc)
        
        #result = morph.parse(word2proc)[0].normal_form
        #result_non_latin.append()
#morph.parse('краску')[0].normal_form

[[(0, 'серить'), (1, 'уильямс')], [(0, 'расселла'), (1, 'уэстбрук')], [(0, 'леброн'), (1, 'джеймс')], [(0, 'мишель'), (1, 'ви')], [(0, 'яннис'), (1, 'адетокумбо')], [(0, 'кевин'), (1, 'дюрант')], [(0, 'наоми'), (1, 'осака')], [(0, 'лик'), (1, 'мартенс')], [(0, 'тайгера'), (1, 'вудс')], [(0, 'килиан'), (1, 'мбапп')], [(0, 'джейсон'), (1, 'дэя')], [(0, 'пол'), (1, 'джордж')], [(0, 'кайри'), (1, 'ирвинга')], [(0, 'рорь'), (1, 'макилрой')], [(0, 'криштиану'), (1, 'роналда')], [(0, 'найдж'), (1, 'хьюстон')], [(0, 'рафаэль'), (1, 'надаль')], [(0, 'холодный'), (1, 'погода')], [(0, 'тёплый'), (1, 'погода')], [(1, 'футбол'), (3, 'и')], [], [], [], [(1, 'в'), (2, 'твой'), (3, 'стиль')], [], [], [(0, 'чили')], [(0, 'южный'), (1, 'корея')], [(0, 'великобритания')], [(0, 'турция')], [(0, 'словакия')], [(0, 'норвегия')], [(0, 'португалия')], [(0, 'страна')], [(0, 'испания')], [(0, 'греция')], [(0, 'нидерланды')], [(0, 'франция')], [(0, 'сша')], [(0, 'австралия')], [(0, 'хорватия')], [(0, 'словения')

In [30]:
# now we need to make it look like before
print(np.shape(result_non_latin))

(16695,)


In [35]:
latin = [rec[0] for rec in data_proc] 
print(len(latin))
latin[:10]

16695


[[], [], [], [], [], [], [], [], [], []]

In [40]:
right_order_list = []

for i in range(len(data_list)):
    # get initial sentence length
    sent_len = len(data_list[i].split(' '))
    # create new version of sentence
    sent = [''] * sent_len
    # fill latins and non-latins
    for word_pos in latin[i]:
        sent[word_pos[0]] = word_pos[1]
    for word_pos in result_non_latin[i]:
        sent[word_pos[0]] = word_pos[1]
    # save the result
    right_order_list.append(sent)
    
right_order_list[100:110]

[['сумка'],
 ['платье'],
 ['бутсы'],
 ['шлёпанец'],
 ['нижний', 'бельё'],
 ['комбинезон'],
 ['перчатка', 'и', 'варежка'],
 ['рукав'],
 ['спортивный', 'костюм'],
 ['верх']]

In [45]:
# into sentences
right_order_sentences = [
    ' '.join(l)
    for l in right_order_list
]

right_order_sentences[300:320]

['шлёпанец Mercurial',
 'шлёпанец Nike Dunk',
 'шлёпанец Pegasus',
 'шлёпанец Nike Free RN',
 'шлёпанец Roshe',
 'шлёпанец Air Max 720',
 'шлёпанец Vomero',
 'шлёпанец ',
 'шлёпанец Jordan 1',
 'сумка Air Max 1',
 'сумка Jordan 5',
 'сумка Huarache',
 'сумка Nike SuperRep',
 'сумка Air Max 90',
 'сумка VaporMax',
 'сумка Infinity Run',
 'сумка Structure',
 'сумка Air Max 95',
 'сумка Air Max 97',
 'сумка Air Force 1']

In [49]:
# save results
res = pd.Series(right_order_sentences, name='SEARCHTERMS')
print(res.shape)
res.head()

(16695,)


0       серить уильямс
1    расселла уэстбрук
2        леброн джеймс
3            мишель ви
4     яннис адетокумбо
Name: SEARCHTERMS, dtype: object

In [48]:
res.to_csv('pymorphy2.csv')

In [46]:
data_list[300:320]

['Шлепанцы Mercurial',
 'Шлепанцы Nike Dunk',
 'Шлепанцы Pegasus',
 'Шлепанцы Nike Free RN',
 'Шлепанцы Roshe',
 'Шлепанцы Air Max 720',
 'Шлепанцы Vomero',
 'Шлепанцы Blazer',
 'Шлепанцы Jordan 1',
 'Сумки Air Max 1',
 'Сумки Jordan 5',
 'Сумки Huarache',
 'Сумки Nike SuperRep',
 'Сумки Air Max 90',
 'Сумки VaporMax',
 'Сумки Infinity Run',
 'Сумки Structure',
 'Сумки Air Max 95',
 'Сумки Air Max 97',
 'Сумки Air Force 1']